In [1]:

from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import cifar100
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from keras.losses import categorical_crossentropy
from keras.utils import print_summary, to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.layers import Input, Lambda
from keras import backend as K
from scipy.linalg import sqrtm

import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:

BATCH_SIZE = 100
NUM_CLASSES = 100
EPOCHS = 1
INIT_DROPOUT_RATE = 0.5
MOMENTUM_RATE = 0.9
INIT_LEARNING_RATE = 0.01
L2_DECAY_RATE = 0.0005
CROP_SIZE = 32
num_labels = 100
batch_size = 100
epochs = 1

In [3]:
classes = 100
#print(current_path)
#model_name = 'cifar100.h5'
(x_train, y_train) , (x_test, y_test) = cifar100.load_data()
print('x_train_dims : ' , x_train.shape)
print('x_test_dims : ', x_test.shape)
print('y_train_dims : ', y_train.shape)
print('y_test_dims : ', y_test.shape)

print( 'number of training examples available : ', x_train.shape[0])
print('number of testing examples available : ', x_test.shape[0])

y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

x_train_dims :  (50000, 32, 32, 3)
x_test_dims :  (10000, 32, 32, 3)
y_train_dims :  (50000, 1)
y_test_dims :  (10000, 1)
number of training examples available :  50000
number of testing examples available :  10000


In [4]:
print('x_train_dims : ' , x_train.shape)
print('x_test_dims : ', x_test.shape)
print('y_train_dims : ', y_train.shape)
print('y_test_dims : ', y_test.shape)
print(x_train.shape[1:])

x_train_dims :  (50000, 32, 32, 3)
x_test_dims :  (10000, 32, 32, 3)
y_train_dims :  (50000, 100)
y_test_dims :  (10000, 100)
(32, 32, 3)


In [5]:
def neg_hscore(x):
    """
        negative hscore calculation
        """
    f = x[0]
    g = x[1]
    f0 = f - K.mean(f, axis = 0)
    g0 = g - K.mean(g, axis = 0)
    corr = tf.reduce_mean(tf.reduce_sum(tf.multiply(f0, g0), 1))
    cov_f = K.dot(K.transpose(f0), f0) / K.cast(K.shape(f0)[0] - 1, dtype = 'float32')
    cov_g = K.dot(K.transpose(g0), g0) / K.cast(K.shape(g0)[0] - 1, dtype = 'float32')
    return - corr + tf.trace(K.dot(cov_f, cov_g)) / 2

In [6]:
fdim = 100
gdim = fdim


input_x = Input(shape = x_train.shape[1:])
zero1 = ZeroPadding2D(4, input_shape=x_train.shape[1:])(input_x)
conv1 = Conv2D(384, kernel_size=(3, 3),
               input_shape=x_train.shape[1:],
               padding='same', kernel_regularizer=l2(0.01))(zero1)
ac1 = Activation('elu')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2), padding='same')(ac1)
f1 = Dropout(0.5)(pool1)

conv2 = Conv2D(384, (1, 1), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f1)
conv3 = Conv2D(384, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv2)
conv4 = Conv2D(640, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv3)
conv5 = Conv2D(640, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv4)
ac2 = Activation('elu')(conv5)
pool2 = MaxPooling2D(pool_size=(2,2), padding='same')(ac2)
f2 = Dropout(0.5)(pool2)


conv6 = Conv2D(640, (3, 3), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f2)
conv7 = Conv2D(768, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv6)
conv8 = Conv2D(768, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv7)
conv9 = Conv2D(768, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv8)
ac3 = Activation('elu')(conv9)
pool3 = MaxPooling2D(pool_size=(2,2), padding='same')(ac3)
f3 = Dropout(0.5)(pool3)

conv10 = Conv2D(768, (1, 1), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f3)
conv11 = Conv2D(896, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv10)
conv12 = Conv2D(896, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv11)
ac4 = Activation('elu')(conv12)
pool4 = MaxPooling2D(pool_size=(2,2), padding='same')(ac4)
f4 = Dropout(0.5)(pool4)


conv13 = Conv2D(896, (3, 3), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f4)
conv14 = Conv2D(1024, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv13)
conv15 = Conv2D(1024, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv14)
ac5 = Activation('elu')(conv15)
pool5 = MaxPooling2D(pool_size=(2,2), padding='same')(ac5)
f5 = Dropout(0.5)(pool5)

conv16 = Conv2D(1024, (1, 1), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f5)
conv17 = Conv2D(1152, (2, 2), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(conv16)
ac6 = Activation('elu')(conv17)
pool6 = MaxPooling2D(pool_size=(2,2), padding='same')(ac6)
f6 = Dropout(0.5)(pool6)

conv18 = Conv2D(1152, (1, 1), padding='same', kernel_regularizer=l2(L2_DECAY_RATE))(f6)
ac7 = Activation('elu')(conv18)
pool7 = MaxPooling2D(pool_size=(2,2), padding='same')(ac7)
f7 = Dropout(0.5)(pool7)

f = Flatten()(f7)
f = Dense(fdim)(f)

input_y = Input(shape = (num_labels, ))
g = Dense(gdim)(input_y)
g = Dropout(0.5)(g)

loss = Lambda(neg_hscore)([f, g])

model = Model(inputs = [input_x, input_y], outputs = loss)

opt = keras.optimizers.rmsprop(lr=0.0001, decay = 1e-6)

model.compile(optimizer=opt, loss = lambda y_true,y_pred: y_pred)

model.fit([x_train, y_train],
          np.zeros([y_train.shape[0], 1]),
          batch_size = batch_size,
          epochs = epochs,
          validation_data=([x_test, y_test], np.zeros([y_test.shape[0], 1])))

model_f = Model(inputs = input_x, outputs = f)
model_g = Model(inputs = input_y, outputs = g)

f_test = model_f.predict(x_test)
f_test0 = f_test - np.mean(f_test, axis = 0)
g_val = model_g.predict(np.eye(100))
py = np.mean(y_train, axis = 0)
g_val0 = g_val - np.matmul(py, g_val)  # get zero-mean g(Y)
# BUG fixed
pygx = py * (1 + np.matmul(f_test0, g_val0.T))
acc = np.mean(np.argmax(pygx, axis = 1) == np.argmax(y_test, axis = 1))
print(acc)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
  400/50000 [..............................] - ETA: 2:20:45 - loss: 12.0586

KeyboardInterrupt: 